In [2]:
# Importing required modules
!pip3 install PyMuPDF pdfplumber


^C
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_internal/build_env.py", line 18, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/

In [8]:
# import fitz  # PyMuPDF

# pdf_file = '/Users/vishmiherath/Documents/FYP/Legal Doc/English_Version.pdf'  # Replace with your PDF file path
# doc = fitz.open(pdf_file)

# # Analyze each page
# for page_num in range(len(doc)):
#     page = doc.load_page(page_num)
#     text = page.get_text()  # Extract text content
#     blocks = page.get_text("blocks")  # Get text blocks with positions
#     images = page.get_images(full=True)  # Check for images

#     print(f"Page {page_num + 1}:")
#     print(f"  - Text Length: {len(text)} characters")
#     print(f"  - Number of Text Blocks: {len(blocks)}")
#     print(f"  - Number of Images: {len(images)}")
#     print(f"  - First 300 characters: {text[:300]}\n")
    
# doc.close()
import fitz  # PyMuPDF

pdf_path = "Legal Doc/English_Version.pdf"
doc = fitz.open(pdf_path)
text = doc[3].get_text("text")  # Page 4 is at index 3 (since index is 0-based)
print(f"Page 4 Content:\n{text}\n")


Page 4 Content:
4 A 
I ෙකොටස : (I) ෙඡදය -  ශී ලංකා පජාතාන්තික සමාජවාදී ජනරජෙය් අති විෙශෂ ගැසට් පතය - 2021.07.08 
PART  I : Sec (I) –  GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIALIST REPUBLIC OF SRI LANKA – 08.07.2021 
PART II 
 
 
Land Subdivision, Amalgamation and Development 
 
 
4. 
(1) No person shall carry out or engage in any Development Activity related to subdivision, amalgamation or 
development of lands without a Development Permit obtained from the Relevant Authority. 
 
(2)  In order to obtain a Development Permit, a Developer or his Agent shall make an application to the Relevant 
Authority according to Form B as set out in Schedule 1 herein, together with Annexure I and II of Form B 
and written declarations as prescribed in Annexure III and IV of Form B endorsed by a Qualified Person as 
set out in Schedule 3 herein.   
 
 (3) In a case where it is deemed necessary by the Relevant Authority, the Developer or his Agent shall submit an 
indemnity for the proposed land su

In [15]:
import fitz  # PyMuPDF
import pdfplumber
import re

pdf_path = "Legal Doc/English_Version.pdf"

# Define the header text to remove (including Sinhala)
header_text = """ෙකොටස : (I) ෙඡදය - ශී ලංකා පජාතාන්තික සමාජවාදී ජනරජෙය් අති විෙශෂ ගැසට් පතය - 2021.07.08 PART I : Sec (I) – GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIALIST REPUBLIC OF SRI LANKA – 08.07.2021"""

# Function to clean header/footer from text (with regex)
def clean_text(text, header):
    if text is None:
        return ""
    # Clean hidden characters and extra spaces
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    header_cleaned = re.sub(r'\s+', ' ', header)  # Clean header similarly
    return "\n".join([line for line in text.split("\n") if header_cleaned not in re.sub(r'\s+', ' ', line)])

# Save the cleaned output to a file
output_file = "cleaned_output.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    # Extract text from each page (excluding header/footer)
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")
        if text:
            cleaned_text = clean_text(text, header_text)  # Clean header/footer
            f.write(f"Cleaned Page {page_num + 1}:\n{cleaned_text}\n\n")
        else:
            f.write(f"No text extracted from page {page_num + 1}\n")

    # Extract tables from each page (excluding header/footer)
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            tables = page.extract_tables()
            if tables:
                for table in tables:
                    # Clean rows of the table by removing header/footer from each cell
                    cleaned_table = []
                    for row in table:
                        cleaned_row = [clean_text(cell, header_text) for cell in row if cell]
                        cleaned_table.append(cleaned_row)
                    for row in cleaned_table:
                        f.write(f"{row}\n")
            else:
                f.write(f"No tables extracted from page {page_num + 1}\n")

    # Extract form pages (pages 45 to 100) and clean header/footer
    form_pages = range(45, 101)  # Pages 45 to 100
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in form_pages:
            page = pdf.pages[page_num - 1]  # pdfplumber uses 0-based index
            text = page.extract_text()
            if text:
                cleaned_text = clean_text(text, header_text)  # Clean header/footer
                f.write(f"Cleaned Form Page {page_num}:\n{cleaned_text}\n\n")
            else:
                f.write(f"No text extracted from form page {page_num}\n")

print(f"Output saved to {output_file}")


Output saved to cleaned_output.txt


In [16]:
normal_text_pages = []  # List to store normal text from pages
extracted_tables = []   # List to store tables
form_pages_text = []    # List to store form page text

# Extract normal text from all pages and clean
doc = fitz.open(pdf_path)
for page_num in range(len(doc)):
    text = doc[page_num].get_text("text")
    if text:
        cleaned_text = clean_text(text, header_text)
        normal_text_pages.append(f"Cleaned Page {page_num + 1}:\n{cleaned_text}")
    else:
        normal_text_pages.append(f"No text extracted from page {page_num + 1}")

# Extract tables from all pages and clean
with pdfplumber.open(pdf_path) as pdf:
    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        tables = page.extract_tables()
        if tables:
            for table in tables:
                cleaned_table = []
                for row in table:
                    cleaned_row = [clean_text(cell, header_text) for cell in row if cell]
                    cleaned_table.append(cleaned_row)
                extracted_tables.append(cleaned_table)
        else:
            extracted_tables.append(f"No tables extracted from page {page_num + 1}")

# Extract form pages (pages 45 to 100) and clean
form_pages = range(45, 101)  # Pages 45 to 100
with pdfplumber.open(pdf_path) as pdf:
    for page_num in form_pages:
        page = pdf.pages[page_num - 1]  # pdfplumber uses 0-based index
        text = page.extract_text()
        if text:
            cleaned_text = clean_text(text, header_text)
            form_pages_text.append(f"Cleaned Form Page {page_num}:\n{cleaned_text}")
        else:
            form_pages_text.append(f"No text extracted from form page {page_num}")

# Now the data is stored in:
# normal_text_pages (list of cleaned text from all pages)
# extracted_tables (list of extracted tables)
# form_pages_text (list of cleaned form pages)

